In [1]:
import cv2
import pafy
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.models import model_from_json 
from matplotlib import pyplot as plt
import tkinter
from tkinter import  *

In [2]:
#Loading the model
model= model_from_json(open("./modeladam.json","r").read())
model.load_weights('./modeladam.h5')

emotions = ('angry','disgust','fear','happy','sad','surprise','neutral')

face_cascade= cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

#Function for FED to work on front cam
def fc():
    #Starting the feed from the front cam
    cap=cv2.VideoCapture(0)
    while(True):
        ret,img=cap.read()    
        
        #Converting the input stream to grayscale
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        #Detecting the faces from the grayscale stream
        faces= face_cascade.detectMultiScale(gray,1.3,5)
        
        #Calssifying the expression on the face and tagging the expression near the face
        for(x,y,w,h) in faces:
            
            #crop the face
            face = img[int(y):int(y+h), int(x):int(x+w)]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            #resize the face
            face = cv2.resize(face, (48, 48))
            
            #Reshaping and Scaling
            img_pixels=image.img_to_array(face)
            img_pixels=np.expand_dims(img_pixels, axis=0)
            img_pixels /=255
            
            #Detecting the expression in the face
            predictions=model.predict(img_pixels)        
            max_index = np.argmax(predictions[0])        
            emotion=emotions[max_index]
            
            #Tagging the expression
            cv2.putText(img,emotion,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
        
        #Showing the real time feed with the expression tagged along the face
        cv2.imshow('img',img)
        if (cv2.waitKey(10))==27:
            break
    cap.release()
    cv2.destroyAllWindows()
    
#Function for FED to work on youtube videos
def yt():
    #Getting the video's url
    url=urlv.get()
    video=pafy.new(url)
    
    #Getting the best video quality of the video
    best=video.getbest(preftype="mp4")
    
    #Streamming the video
    cap=cv2.VideoCapture(best.url)
    
    while(True):
        ret,img=cap.read()    
        
        #Converting the input stream to grayscale
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        #Detecting the faces from the grayscale stream
        faces= face_cascade.detectMultiScale(gray,1.3,5)
        
        #Calssifying the expression on the face and tagging the expression near the face
        for(x,y,w,h) in faces:
            
            #crop the face
            face = img[int(y):int(y+h), int(x):int(x+w)]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            #resize the face
            face = cv2.resize(face, (48, 48))
            
            #Reshaping and resizing the face
            img_pixels=image.img_to_array(face)
            img_pixels=np.expand_dims(img_pixels, axis=0)
            img_pixels /=255
           
            #Detecting the expression in the face
            predictions=model.predict(img_pixels)        
            max_index = np.argmax(predictions[0])        
            emotion=emotions[max_index]
            
            #Tagging the expression
            cv2.putText(img,emotion,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
        
        #Showing the real time feed with the expression tagged along the face
        cv2.imshow('img',img)
        if (cv2.waitKey(10))==27:
            break
    cap.release()
    cv2.destroyAllWindows()
    urlv.set("")

#Function to for FED to work on pics    
def localpic():
    
    #Getting the file path
    filepath=lad.get()
   
    #Loading the image
    img=image.load_img(filepath,grayscale=True,target_size=(48,48))
    
    #Preprocessing the image
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    x=x/255
    
    #Detecting the expression
    detect= model.predict(x)
    
    #Printing the graph of the various expression recorded from the image
    y_pos = np.arange(len(emotions))    
    plt.bar(y_pos, detect[0], align='center', alpha=0.5)
    plt.xticks(y_pos, emotions)
    plt.ylabel('percentage')
    plt.title('emotion')    
    plt.show()
        
    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);
    
    #Printing the image
    plt.gray()
    plt.imshow(x)   
    plt.show()

#Creating the UI
master=tkinter.Tk()
master.title("Facial Expression Detector")
master.geometry("600x600")
urlv=tkinter.StringVar()
lad=tkinter.StringVar()

lab=tkinter.Label(master,text="FED on Front Cam", font = ('calibre',10,'normal'))
lab.grid(row=0,column=3)
button1=tkinter.Button(master, text="FED on front cam", command=fc)
button1.grid(row=1,column=3)

lab1=tkinter.Label(master, text="FED on YOUTUBE (Enter the link of the video)",font=('calibre',10, 'bold'))
lab1.grid(row=2,column=3)
ent1=tkinter.Entry(master, textvariable=urlv,font=('calibre',10, 'bold'))
ent1.grid(row=3,column=3)
button2=tkinter.Button(master, text="FED on Youtube", command=yt)
button2.grid(row=4,column=3)

lab2=tkinter.Label(master, text="FED on image (Enter the path of the image saved locally)",font=('calibre',10, 'bold'))
lab2.grid(row=5,column=3)
ent2=tkinter.Entry(master, textvariable=lad,font=('calibre',10, 'bold'))
ent2.grid(row=6,column=3)
button3=tkinter.Button(master, text="FED on image", command=localpic)
button3.grid(row=7,column=3)

master.mainloop()